<a href="https://colab.research.google.com/github/yf591/llm-toolkit/blob/main/Universal_LLM_GUI_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 必要なライブラリーのインストールおよびインポート

In [8]:
from google.colab import output

!pip install transformers torch gradio

output.clear()

In [2]:
import torch  # PyTorchライブラリをインポート（テンソル演算、GPU使用などに必要）
from transformers import AutoModelForCausalLM, AutoTokenizer # Hugging Face Transformersライブラリから必要なクラスをインポート（言語モデル、トークナイザー）
import gradio as gr # Gradioライブラリをインポート（Web UI作成用）

In [6]:
# Check GPU availability
!nvidia-smi

print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"Current Device: {torch.cuda.get_device_name(0)}")

Thu Jan 23 18:25:04 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   77C    P0              37W /  72W |  15737MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## ModelとTokenizerをLoad

In [3]:
# モデルとトークナイザーをロード

#@markdown ### ◆使用するモデルの名前 (Hugging FaceのモデルID)
model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"  #@param {type:"string"}

# トークナイザーをロード
tokenizer = AutoTokenizer.from_pretrained(model_name)
# トークナイザー：テキストをモデルが処理できる数値（トークン）に変換する役割
# from_pretrained(): Hugging Faceから事前学習済みのトークナイザーをダウンロードしてロード

# モデルをロード
model = AutoModelForCausalLM.from_pretrained(
    model_name,                 # ロードするモデルの名前
    torch_dtype=torch.float16,   # モデルのデータ型をfloat16に設定 (メモリ使用量削減)
    device_map="auto"           # 利用可能なデバイス（GPU/CPU）に自動配置
)
#@markdown #### - **AutoModelForCausalLM**: 因果言語モデル (テキスト生成など) のためのモデルクラス
#@markdown #### - **from_pretrained()**: Hugging Faceから事前学習済みのモデルをダウンロードしてロード


#@markdown ---
#@markdown **※補足：torch_dtype=torch.float16 について**

#@markdown float16 (半精度浮動小数点数) を使用することで、float32 (単精度浮動小数点数) の半分以下のメモリでモデルを格納できます。
#@markdown これにより、メモリ容量が限られた環境（例：Colabの無料枠など）でも大規模モデルを使用しやすくなります。
#@markdown ただし、精度が若干低下する可能性があるので、必要に応じてfloat32を使用することも検討してください。

#@markdown ---
#@markdown **※補足：device_map="auto" について**

#@markdown "auto" を指定することで、PyTorchが自動的に最適なデバイスを選択します。
#@markdown GPUが利用可能な場合はGPUが選択され、そうでない場合はCPUが使用されます。
#@markdown GPUを使用する場合は、Colabのランタイムタイプを「GPU」に設定していることを確認してください。

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/7.39G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

'\n補足：torch_dtype=torch.float16 について\n\nfloat16 (半精度浮動小数点数) を使用することで、float32 (単精度浮動小数点数) の半分以下のメモリでモデルを格納できます。\nこれにより、メモリ容量が限られた環境（例：Colabの無料枠など）でも大規模モデルを使用しやすくなります。\nただし、精度が若干低下する可能性があるので、必要に応じてfloat32を使用することも検討してください。\n\n補足：device_map="auto" について\n\n"auto" を指定することで、PyTorchが自動的に最適なデバイスを選択します。\nGPUが利用可能な場合はGPUが選択され、そうでない場合はCPUが使用されます。\nGPUを使用する場合は、Colabのランタイムタイプを「GPU」に設定していることを確認してください。\n'

`model_name` を変更することで、Hugging Faceにある様々な言語モデルを簡単に試すことができます。

例えば：

1. GPT系のモデル
   - "gpt2"
   - "EleutherAI/gpt-j-6B"

2. Llama系
   - "meta-llama/Llama-2-7b"
   - "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

3. その他の多様なモデル
   - "google/flan-t5-large"
   - "bigscience/bloom-560m"

**主な注意点**:
- モデルによって推論方法や引数が若干異なる可能性
- GPUメモリ要件が変わる
- 性能や特性は大きく異なる

**基本的なコードフレームワークは同じで、`model_name`だけ変更すれば新しいモデルを試せます。**

## テキスト生成をおこなう関数の定義

In [4]:
def generate_text(prompt, max_input_length=1024, max_output_length=200):
    """テキスト生成を行う関数

    Args:
        prompt (str): 入力テキスト文字列
        max_input_length (int): 入力テキストの最大長 (デフォルト: 1024)
        max_output_length (int): 生成するテキストの最大長 (デフォルト: 200)

    Returns:
        str: 生成されたテキスト文字列
    """
    # 入力テキストをトークン化し、PyTorchテンソルに変換
    inputs = tokenizer(
        prompt,
        truncation=True,        # 入力テキストがmax_lengthを超えた場合に切り詰める
        max_length=max_input_length, # 入力テキストの最大長
        return_tensors="pt"     # PyTorchテンソルとして結果を返す
    ).to(model.device)          # テンソルをモデルと同じデバイス（GPUまたはCPU）に移動

    # モデルを使ってテキストを生成
    outputs = model.generate(
        **inputs,            # トークン化された入力をキーワード引数として展開
        max_length=max_output_length # 生成するテキストの最大長
    )

    # 生成されたトークンIDを文字列にデコード
    return tokenizer.decode(
        outputs[0],         # 生成されたトークンIDのリスト（バッチサイズが1の場合）
        skip_special_tokens=True # 特殊トークン（例：<pad>, <s>, </s>）を除外
    )

## GUIの設定

In [ ]:
#@title GUIインターフェイスの設定

interface = gr.Interface(
    fn=generate_text, # 使用する関数（テキスト生成関数）
    inputs=[         # 入力インターフェースの設定
        gr.Textbox(
            label="Prompt",      # テキストボックスのラベル
            lines=10,            # テキストボックスの初期表示行数
            max_lines=20         # テキストボックスの最大行数（スクロールバーが表示される）
        ),
        gr.Slider(
            minimum=128,         # スライダーの最小値
            maximum=2048,        # スライダーの最大値
            value=1024,          # スライダーの初期値
            label="Max Input Length" # スライダーのラベル
        ),
        gr.Slider(
            minimum=50,          # スライダーの最小値
            maximum=500,         # スライダーの最大値
            value=200,           # スライダーの初期値
            label="Max Output Length" # スライダーのラベル
        )
    ],
    outputs=gr.Textbox(
        label="Generated Text", # テキストボックスのラベル
        lines=10,               # テキストボックスの初期表示行数
        max_lines=20            # テキストボックスの最大行数（スクロールバーが表示される）
    )
)

In [ ]:
#@title GUI起動
interface.launch(share=True) # Gradioインターフェースを起動。share=Trueで外部からアクセス可能にする